<a href="https://colab.research.google.com/github/dariianina/ThermoGen/blob/main/CUT_train_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CUT
This new model uses contrastive learning (the hot technique of the moment) to better train unaligned image to image translation (i.e. CycleGAN)

[GitHub](https://github.com/taesungp/contrastive-unpaired-translation) | [Website](http://taesung.me/ContrastiveUnpairedTranslation/)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
if os.path.isdir("/content/drive/My Drive/colab-cut-test/CUT"):
    %cd /content/drive/My\ Drive/colab-cut-test/CUT
else:
    %cd /content/drive/My\ Drive
    #!mkdir colab-cut-test
    %cd ./colab-cut-test/
    #!git clone https://github.com/taesungp/contrastive-unpaired-translation CUT
    %cd CUT

#dependencies
#!pip install dominate visdom GPUtil

/content/drive/My Drive/colab-cut-test/CUT


## Download pre-built datasets

**For custom datasets:** create a folder in `./datasets` and put two folders inside it. `trainA` should contain images from one domain and `trainB` should contain images from another domain.

In [ ]:
#datasets available: apple2orange, summer2winter_yosemite, horse2zebra, monet2photo, cezanne2photo, ukiyoe2photo, vangogh2photo, maps, cityscapes, facades, iphone2dslr_flower, ae_photos, grumpifycat
dataset = "horse2zebra"
#example: download the russian blue cats to grumpy cats dataset
!bash ./datasets/download_cut_dataset.sh $dataset

In [3]:
dataset = "rgb2therm"
#!git clone https://github.com/vision-cidis/CIDIS-dataset.git

In [10]:
!cp -r ./CIDIS-dataset/dataset/visible/train ./datasets/rgb2therm/trainA
!cp -r ./CIDIS-dataset/dataset/visible/test ./datasets/rgb2therm/testA
!cp -r ./CIDIS-dataset/dataset/thermal/train ./datasets/rgb2therm/trainB
!cp -r ./CIDIS-dataset/dataset/thermal/test ./datasets/rgb2therm/testB

RGB to greyscale

## Train


In [4]:
dataset_path = './datasets/' + dataset
project_name = dataset + '_CUT'
load_size = 286
crop_size = 256

In [4]:
!python train.py --max_dataset_size 4 \
--lr 0.002 \
--input_nc 1 \
--output_nc 1 \
--dataroot $dataset_path \
--name $project_name \
--CUT_mode CUT \
--load_size $load_size \
--crop_size $crop_size \
--display_id 0
# --continue_train False \

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
                    beta1: 0.5                           
                    beta2: 0.999                         
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/rgb2therm          	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 0                             	[default: None]
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost      

In [11]:
!ls ./checkpoints/rbd2therm_CUT/

ls: cannot access './checkpoints/rbd2therm_CUT/': No such file or directory


In [19]:
from google.colab import files
files.download('./checkpoints/rgb2therm_CUT/latest_net_F.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Test



*   `--num_test`: how many test images to produce
*   `--direction`: `AtoB` or `BtoA`





In [10]:
!python test.py --dataroot ./datasets/rgb2therm/ \
 --direction AtoB \
 --max_dataset_size 4 \
 --name rgb2therm_CUT \
 --load_size 286 \
 --crop_size 256 \
 --input_nc 1 \
 --output_nc 1 \
--CUT_mode CUT

----------------- Options ---------------
                 CUT_mode: CUT                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/rgb2therm/         	[default: placeholder]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
               easy_label: experiment_name               
                    epoch: latest                        
                     eval: False                         
        flip_equivariance: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: xavier                        
                 input_nc: 1                             	[defaul

In [9]:
!cp -r ./datasets/rgb2therm/trainB/ ./datasets/rgb2therm/testB